In [1]:
import yfinance as yf
import pandas as pd
import mplfinance as mpf
import numpy as np

# 获取 SPY 最近 30 天日线数据
ticker = "SPY"
spy = yf.Ticker(ticker)
data = spy.history(period="1mo")  # 获取最近 1 个月数据

# 计算 Bull Trend Bar 逻辑
def is_bull_trend_bar(row):
    """ 判断当前 K 线是否符合 Bull Trend Bar 的定义 """
    body = row["Close"] - row["Open"]
    upper_shadow = row["High"] - max(row["Open"], row["Close"])
    lower_shadow = min(row["Open"], row["Close"]) - row["Low"]
    
    # Bull Trend Bar 逻辑：主体大于 80%（即最小占比）
    body_size = abs(body)
    total_range = row["High"] - row["Low"]
    
    return (row["Close"] > row["Open"]) and (body_size / total_range >= 0.8)

# 筛选 Bull Trend Bar
data["BullTrendBar"] = data.apply(is_bull_trend_bar, axis=1)

# 找出 Bull Trend Bar 位置
bull_bars = data[data["BullTrendBar"]]

# 绘制 K 线图
fig, ax = mpf.plot(
    data, 
    type="candle", 
    style="charles",
    volume=False, 
    figsize=(10,6),
    returnfig=True
)

# 添加绿色向上箭头标注
for idx in bull_bars.index:
    ax[0].annotate("↑", 
                   (idx, bull_bars.loc[idx, "Low"] - 0.5), 
                   color="green", 
                   fontsize=12, 
                   ha="center")

# 显示图表
mpf.show()

Failed to get ticker 'SPY' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
$SPY: possibly delisted; no price data found  (period=1mo)


TypeError: Expect data.index as DatetimeIndex

In [ ]:
from ib_insync import *

# 连接到 IBKR Paper Trading 账户（端口 7497）
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

# 检查连接状态
if ib.isConnected():
    print("✅ 成功连接到 IBKR Paper Trade API")
else:
    print("❌ 连接失败，请检查 TWS / IB Gateway 设置")